In [9]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None)

# Import the data set VDL_Norfolk_Island_Penal_Colony_pp.csv
with open('2. VDL_Norfolk_Island_Penal_Colony_pp.csv', 'r') as file:
    data = pd.read_csv(file)

data

,trial_id,register_name,def_firstname,def_othername,def_sex,def_surname,notes,offence,offence_pp,sentence,sentence_pp,trial_date,trial_place,verdict,line_number,fas_id,date_arrived_aus,manual_source,police_number,sentence_years,sentence_years_pp,pris_ht,pris_ht_pp,def_age,def_age_pp,def_literacy,def_literacy_pp,def_religion,def_religion_pp,marital_status,marital_status_pp,children_nr,cash_sav,cash_sav_pp,occupation,occupation_pp,birth_country,coloffence_info,prev_transp,priorconvictions_text,depni_date,depni_ship,offence_ni,probat_date,tl_date,death_notes,death_in_custody_pp,probat_dur,ship,trial_court,date_1stkconv,offence_1stkconv,trialplace_1stkconv,sentence_1stkconv,sentduryears_1stkconv,court_1stkconv,arrivni_date
0,695735,VDL_NIPC,WILLIAM,NaN,MALE,ADAMS,BEGIN FILE NOTES: [SENTENCE COMMENT] [DEPARTED...,RAPE (ANN PEPPERS),RAPE,LIFE,99.0,1844-03-09,AYLESBURY A,NaN,MB1,NI100001,1844-11-09,CON33/1/83,19167,99,99.0,5 5,165.10,25,25.0,R,True,P,P,S,S,NaN,NaN,NaN,FARM LABOURER,FARM,NR BUCKINGHAM,.,NaN,[INFORMATION IN 'STATED THIS OFFENCE' BOX] OF ...,1847-04-08,PESTONJEE BOMANJEE,NONE,9/5/1848,7/3/1854,NaN,False,30MTHS,AGINCOURT,NaN,1844-03-09,RAPE (ANN PEPPERS),AYLESBURY A,LIFE,99,NaN,NaN
1,695736,VDL_NIPC,WILLIAM,NaN,MALE,ALDRIDGE,BEGIN FILE NOTES: [SENTENCE COMMENT] [DEPARTED...,RAPE,RAPE,LIFE,99.0,1844-02-28,HERTFORD A,NaN,MB2,NI100002,1844-11-09,CON33/1/83,19168,99,99.0,5 5,165.10,44,44.0,R,True,P,P,S,S,NaN,NaN,NaN,BOATMAN,BOATMAN,"PADDINGTON, LONDON",.,NaN,[INFORMATION IN 'STATED THIS OFFENCE' BOX] OF ...,1847-04-08,PESTONJEE BOMANJEE,24/8/46 ADMON MISC,9/5/1848,1/8/1854,NaN,False,30MTHS,AGINCOURT,NaN,1844-02-28,RAPE,HERTFORD A,LIFE,99,NaN,NaN
2,695737,VDL_NIPC,WILLIAM,NaN,MALE,ALLEN,BEGIN FILE NOTES: [SENTENCE COMMENT] [DEPARTED...,BURGLARY (,BURGLARY,10YRS,10.0,1843-08-03,WARWICK,NaN,MB3,NI100003,1844-11-09,CON33/1/83,19169,10,10.0,5 6 1/4,167.64,31,31.0,B,True,P,P,S,S,NaN,NaN,NaN,BRASS FOUNDER,BRASS,NR. BIRMINGHAM,.,NaN,[INFORMATION IN 'STATED THIS OFFENCE' BOX] PRO...,1846-12-30,LADY FRANKLIN,NONE,9/12/1847,20/11/1849,NaN,False,24M,AGINCOURT,NaN,1843-08-03,BURGLARY (,WARWICK,10YRS,10,NaN,NaN
3,695738,VDL_NIPC,THOMAS,GOLDSTRAW RUPERT,MALE,ARKINSTALL,BEGIN FILE NOTES: [SENTENCE COMMENT] [DEPARTED...,BURGLARY (CLOTH),BURGLARY,10YRS,10.0,1844-03-09,STAFFORD A,NaN,MB4,NI100004,1844-11-09,CON33/1/83,19170,10,10.0,5 7,170.18,49,49.0,BL,False,P,P,M,M,11,NaN,NaN,SHOEMAKER,SHOEMAKER,NR. STAFFORD,.,NaN,[INFORMATION IN 'STATED THIS OFFENCE' BOX] PRO...,1846-12-30,NaN,NONE,9/12/1847,2/10/1849,NaN,False,24M,AGINCOURT,NaN,1844-03-09,BURGLARY (CLOTH),STAFFORD A,10YRS,10,NaN,NaN
4,695739,VDL_NIPC,JAMES,NaN,MALE,ASPLIN,BEGIN FILE NOTES: [SENTENCE COMMENT] [DEPARTED...,CUTTING AND MAIMING,CUTTING,15YRS,15.0,1844-03-14,BEDFORD A,NaN,MB5,NI100005,1844-11-09,CON33/1/83,19171,15,15.0,5 6,167.64,23,23.0,RL,False,P,P,S,S,NaN,NaN,NaN,FARM LABOURER. CAN PLOUGH,FARM,NR BEDFORD,.,NaN,[INFORMATION IN 'STATED THIS OFFENCE' BOX] ON ...,1846-12-30,LADY FRANKLIN,NONE,9/12/1847,23/9/1850 (LOST ONCE),NaN,False,24M,AGINCOURT,NaN,1844-03-14,CUTTING AND MAIMING,BEDFORD A,15YRS,15,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6467,714854,VDL_NIPC,JAMES,NaN,NaN,MCNAB,[SECOND CRIME]SENT OUT UNDER FIRST OFFENCE[CON...,NO DATA,NO,NaN,NaN,1/1/1970,NaN,NaN,FBC95,NI400095,1/1/1970,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6468,714855,VDL_NIPC,JOHN,NaN,NaN,MCNULTY,[SECOND CRIME]SENT OUT UNDER FIRST OFFENCE[CON...,HIGHWAY ROBBERY,HIGHWAY,"3YRS, FROM DEATH",3.0,1833-05-23,SYDNEY,NaN,FBC96,NI400096,1/1/1970,35/1/1 P435,581,3,3.0,NaN,NaN,18,18.0,NaN,NaN,ROMAN CATHOLIC,RC,SINGLE,S,NaN,NaN,NaN,BOOT CLOSER,BOOT,AUSTRALIA,NaN,NaN,NaN,NaN,NaN,NOTHING RECORDED,NaN,14 

In [10]:
imputed_value_count = {}

print("1 - Checking: trial_id")
if data['trial_id'].is_unique and data['trial_id'].notnull().all():
    print("    trial_id is unique and not null")
else:
    raise ValueError("    trial_id is not unique or contains null values")

imputed_value_count['trial_id'] = 0

print("2 - Dropped Columns: def_firstname, def_othername, def_surname")
data = data.drop(columns=['def_firstname', 'def_othername', 'def_surname'])


if len(data['register_name'].unique()) == 1:
    data = data.drop(columns=['register_name'])
    print("3 - Dropped Columns: register_name")


if  len(data.dropna(subset=("def_sex"))['def_sex'].unique()) == 1:
    data = data.drop(columns=['def_sex'])
    print("4 - Dropped Columns: def_sex")


data = data.drop(columns=['notes'])
print("5 - Dropped Columns: notes")

# ==========

print("6 - Checking: offence")

# Drop the offence column as it has it has been preprocessed
data = data.drop(columns=['offence'])

most_common_offences = data['offence_pp'].value_counts()

# Print the number of unique offences
print(f"    Number of unique offences in offence_pp: {len(most_common_offences)}")

# For any offence with count < 10, rename it to 'UNKNOWN' (We pretend that we don't know the offence)
for offence, count in most_common_offences.items():
    if count < 10:
        data['offence_pp'] = data['offence_pp'].replace({offence: 'UNKNOWN'})


for remove_offence in ['NO', 'SENT', 'ORDERED', 'COCKATOO', 'STRAIGHT', 
                       'TRANSFERRED', 'FELONY', 'TO', 'X', 'NOT', 'AT', 'ATTEMPTED', 'STREET', 'ILLEGALLY', 'MHLC']:
    data['offence_pp'] = data['offence_pp'].replace({remove_offence: 'UNKNOWN'})
data['offence_pp'] = data['offence_pp'].replace({np.nan: 'UNKNOWN'})

# Unify the offence values
data['offence_pp'] = data['offence_pp'].replace({'HIGHWAY': 'HIGHWAY_ROBBERY', 'HIGHWAYROBBERY': 'HIGHWAY_ROBBERY'})
data['offence_pp'] = data['offence_pp'].replace({'CATTLE': 'CATTLE_STEALING', 'CATTLESTEALING': 'CATTLE_STEALING'})
data['offence_pp'] = data['offence_pp'].replace({'HOUSEBREAKING': 'BURGLARY'})
data['offence_pp'] = data['offence_pp'].replace({'HOUSE': 'BURGLARY'})
data['offence_pp'] = data['offence_pp'].replace({'SHEEP': 'SHEEP_STEALING', 'SHEEPSTEALING': 'SHEEP_STEALING'})
data['offence_pp'] = data['offence_pp'].replace({'HORSE': 'HORSE_STEALING', 'HORSESTEALING': 'HORSE_STEALING'})
data['offence_pp'] = data['offence_pp'].replace({'RECEIVING': 'RECEIVING_STOLEN_GOODS'})
data['offence_pp'] = data['offence_pp'].replace({'LARCENY': 'STEALING'})
data['offence_pp'] = data['offence_pp'].replace({'ABSCONDED': 'ABSCONDING'})
data['offence_pp'] = data['offence_pp'].replace({'FORGING': 'FORGERY'})
data['offence_pp'] = data['offence_pp'].replace({'ASSAULTING': 'ASSAULT'})
data['offence_pp'] = data['offence_pp'].replace({'ROBBING': 'ROBBERY'})


# Define the mapping dictionary
offence_general_map = {
    # STEALING
    'BURGLARY': 'STEALING',
    'STEALING': 'STEALING',
    'CATTLE_STEALING': 'STEALING',
    'SHEEP_STEALING': 'STEALING',
    'HORSE_STEALING': 'STEALING',
    'SHOPLIFTING': 'STEALING',
    
    # ROBBERY
    'HIGHWAY_ROBBERY': 'ROBBERY',
    'ROBBERY': 'ROBBERY',
    
    # ASSAULT
    'ASSAULT': 'ASSAULT',
    'STRIKING': 'ASSAULT',
    'KILLING': 'ASSAULT',
    'WOUNDING': 'ASSAULT',
    'CUTTING': 'ASSAULT',
    'STABBING': 'ASSAULT',
    'PICKING': 'ASSAULT',
    'SHOOTING': 'ASSAULT',
    'UTTERING': 'ASSAULT',
    'MANSLAUGHTER': 'ASSAULT',
    'MURDER': 'ASSAULT',
    
    # SEXUAL
    'RAPE': 'SEXUAL',
    'SODOMY': 'SEXUAL',
    'BESTIALITY': 'SEXUAL',
    
    # FORGERY
    'FORGERY': 'FORGERY',
    'PERJURY': 'FORGERY',
    'PIRACY': 'FORGERY',
    
    # FINANCE
    'EMBEZZLEMENT': 'FINANCE',
    'COINING': 'FINANCE',
    
    # PROCESSING
    'RECEIVING_STOLEN_GOODS': 'PROCESSING',
    'OBTAINING': 'PROCESSING',
    'POSSESSING': 'PROCESSING',
    
    # DAMAGE
    'BREAKING': 'DESTRUCTION',
    'ARSON': 'DESTRUCTION',
    
    # ESCAPE
    'BUSHRANGING': 'ESCAPE',
    'DESERTION': 'ESCAPE',
    'ABSCONDING': 'ESCAPE',
    
    # UNKNOWN
    'UNKNOWN': 'UNKNOWN'
}

data['offence_pp_general'] = data['offence_pp'].map(offence_general_map)
cols = list(data.columns)
offence_index = cols.index('offence_pp')
cols.remove('offence_pp_general')
cols.insert(offence_index, 'offence_pp_general')
data = data[cols] # Reorder the DataFrame

# Print the number of unique offences in the offence_pp column
print(f"    Number of unique offences in offence_pp: {len(data['offence_pp'].unique())}")
# Print the number of unique offences in the offence_pp_general column
print(f"    Number of unique offences in offence_pp_general: {len(data['offence_pp_general'].unique())}")

# Print all unique offences in the offence_general_pp column and their counts
print("    Unique offences in offence_pp_general:")
for offence, count in data['offence_pp_general'].value_counts().items():
    print(f"        {offence}: {count}")

# Print all unique offences in the offence_pp column and their counts
print("    Unique offences in offence_pp:")
for offence, count in data['offence_pp'].value_counts().items():
    print(f"        {offence}: {count}")

assert data['offence_pp_general'].notnull().all()

# Check the number of UNKNOWN values in the offence_pp_general column
unknown_count = data['offence_pp_general'].value_counts().get('UNKNOWN', 0)
imputed_value_count['offence_pp_general'] = unknown_count
unknown_count = data['offence_pp'].value_counts().get('UNKNOWN', 0)
imputed_value_count['offence_pp'] = unknown_count

# ==========

print("7 - Checking: sentence, sentence_pp, sentence_years, and sentence_years_pp")
# Drop the original sentence column as it has been preprocessed
data = data.drop(columns=['sentence', 'sentence_years'])

# Create a new column 'pp_sentence_years' with values = sentence_years_pp if sentence_years_pp is not null, otherwise use sentence_pp
data['pp_sentence_years'] = data['sentence_years_pp'].where(data['sentence_years_pp'].notnull(), data['sentence_pp'])

# Drop null values in the 'pp_sentence_years' column
null_pp_sentence_years_count = data['pp_sentence_years'].isnull().sum()
print(f"    Dropped {null_pp_sentence_years_count} rows with null pp_sentence_years values")
data = data.dropna(subset=['pp_sentence_years'])

# Check if all pp_sentence_years values are numeric and not null
if data['pp_sentence_years'].apply(lambda x: isinstance(x, (int, float)) and not pd.isnull(x)).all():
    print("    All pp_sentence_years values are now numeric and not null")

# Move the pp_sentence_years column to the left of sentence_pp
cols = list(data.columns)
pp_sentence_years_index = cols.index('pp_sentence_years')
sentence_pp_index = cols.index('sentence_pp')
cols.remove('pp_sentence_years')
cols.insert(sentence_pp_index, 'pp_sentence_years')
data = data[cols]  # Reorder the DataFrame

# Drop the original sentence_pp, sentence_years_pp
data = data.drop(columns=['sentence_pp'])
data = data.drop(columns=['sentence_years_pp'])

imputed_value_count['pp_sentence_years'] = 0

# ==========

print("8 - Checking: trial_date")
# Split the trial_date (1844-03-09) into year and month
data['trial_date'] = pd.to_datetime(data['trial_date'], format='%Y-%m-%d', errors='coerce')

# Drop the original trial_date column
null_trial_date_count = data['trial_date'].isnull().sum()
print(f"    Dropped {null_trial_date_count} rows with null trial_date values")
data = data.dropna(subset=['trial_date'])

# Split the trial_date into year and month
data['trial_year'] = data['trial_date'].dt.year
data['trial_month'] = data['trial_date'].dt.month
# Drop the original trial_date column
data = data.drop(columns=['trial_date'])

# Move the trial_year and trial_month columns to the left of trial_place
cols = list(data.columns)
trial_year_index = cols.index('trial_year')
trial_month_index = cols.index('trial_month')
trial_place_index = cols.index('trial_place')
cols.remove('trial_year')
cols.remove('trial_month')
cols.insert(trial_place_index, 'trial_year')
cols.insert(trial_place_index, 'trial_month')
data = data[cols]  # Reorder the DataFrame

# Max and min trial year
min_trial_year = data['trial_year'].min()
max_trial_year = data['trial_year'].max()
print(f"    Min trial year: {min_trial_year}")
print(f"    Max trial year: {max_trial_year}")

imputed_value_count['trial_year'] = 0
imputed_value_count['trial_month'] = 0

# ==========

print("9 - Checking: trial_place")

print(f"    Number of null trial places has been replaced to 'UNKNOWN': {data['trial_place'].isnull().sum()}")
# Replace all null trial places with 'UNKNOWN'
data['trial_place'] = data['trial_place'].replace({np.nan: 'UNKNOWN'})
# # Count the number of unique trial places
# unique_trial_places = data['trial_place'].nunique()
# print(f"    Number of unique trial places: {unique_trial_places}")

# Keep only the top 30 most common trial places
most_common_trial_places = data['trial_place'].value_counts()
for trial_place, count in most_common_trial_places.items():
    if count < 20:
        data['trial_place'] = data['trial_place'].replace({trial_place: 'UNKNOWN'})

trial_place_map = {
    'CCC': 'CENTRAL CRIMINAL COURT',
    'LONDON, MIDDLESEX': 'LONDON',
    'YORK ASSIZES': 'YORKSHIRE',
    'YORKSHIRE A': 'YORKSHIRE',
    'YORKSHIRE ASSIZES': 'YORKSHIRE',
    'LIVERPOOL ASSIZES': 'LIVERPOOL',
    'LIVERPOOL A': 'LIVERPOOL',
    'STAFFORD A': 'STAFFORD',
    'STAFFORD ASSIZES': 'STAFFORD',
    'SYDNEY': 'AUSTRALIA',
    'HOBART': 'AUSTRALIA',
    'DUBLIN, DUBLIN': 'IRELAND',
    'LIMERICK': 'IRELAND',
    'GLASGOW': 'SCOTLAND'
}
data['trial_place'] = data['trial_place'].replace(trial_place_map)

# Print all unique trial places and their counts
print("    Unique trial places:")
for trial_place, count in data['trial_place'].value_counts().items():
    print(f"        {trial_place}: {count}")

# Check the number of UNKNOWN values in the trial_place column
unknown_count = data['trial_place'].value_counts().get('UNKNOWN', 0)
imputed_value_count['trial_place'] = unknown_count

# ==========

print("10 - Dropped Columns: verdict")
data = data.drop(columns=['verdict'])

# ==========

print("11 - Dropped Columns: line_number, fas_id, manual_source")
data = data.drop(columns=['line_number'])
data = data.drop(columns=['fas_id'])
data = data.drop(columns=['manual_source'])
# ==========

print("12 - Dropped Columns: ship")
data = data.drop(columns=['ship'])

# ==========

print("13 - Dropped Columns: police_number")
data = data.drop(columns=['police_number'])

# ==========

print("14 - Checking: pris_ht	pris_ht_pp")
data = data.drop(columns=['pris_ht'])
# For all unrealistic heights, replace them with null
min_height = 100
max_height = 250
data['pris_ht_pp'] = data['pris_ht_pp'].where(data['pris_ht_pp'].between(min_height, max_height), np.nan)

# Drop null values in the 'pris_ht_pp' column
null_pris_ht_pp_count = data['pris_ht_pp'].isnull().sum()
median_height = data['pris_ht_pp'].median()
print(f"    {null_pris_ht_pp_count} rows with null pris_ht_pp values, replaced with the median height {median_height}")
data['pris_ht_pp'] = data['pris_ht_pp'].fillna(median_height)
assert data['pris_ht_pp'].notnull().all()

imputed_value_count['pris_ht_pp'] = null_pris_ht_pp_count

# ==========
print("15 - Checking: def_age, def_age_pp")

data = data.drop(columns=['def_age'])
# For all unrealistic ages, replace them with null
min_age = 0
max_age = 120
data['def_age_pp'] = data['def_age_pp'].where(data['def_age_pp'].between(min_age, max_age), np.nan)
# Drop null values in the 'def_age_pp' column
null_def_age_pp_count = data['def_age_pp'].isnull().sum()
median_age = data['def_age_pp'].median()
print(f"    {null_def_age_pp_count} rows with null def_age_pp values, replaced with the median age {median_age}")
data['def_age_pp'] = data['def_age_pp'].fillna(median_age)
assert data['def_age_pp'].notnull().all()

imputed_value_count['def_age_pp'] = null_def_age_pp_count

# ==========
print("16 - Checking: def_literacy, def_literacy_pp")
data = data.drop(columns=['def_literacy_pp'])

# ['R' 'B' 'BL' 'RL' 'N' 'B WELSH, L ENGLISH' 'SUP' 'NO DATA' nan 'UNKNOWN' 'B IN PORTUGUESE' 'R FRENCH']
# Function mapping to nominal values to numeric values
def literacy_mapping(literacy):
    if literacy in ['N']:
        return 0
    elif literacy in ['RL', 'R FRENCH']:
        return 1
    elif literacy in ['R', 'BL', 'B WELSH, L ENGLISH', 'B IN PORTUGUESE']:
        return 2
    elif literacy in ['B']:
        return 3
    elif literacy in ['SUP']:
        return 4
    elif literacy in ['NO DATA', 'UNKNOWN', np.nan]:
        return np.nan
    else:
        raise ValueError(f"Unknown literacy value: {literacy}")
    
# Apply the mapping function to the 'def_literacy' column
data['def_literacy'] = data['def_literacy'].apply(literacy_mapping)
# Compute the median literacy value
median_literacy = data['def_literacy'].median()
# Replace null values with the median literacy value
print(f"    {data['def_literacy'].isnull().sum()} rows with null def_literacy values, replaced with the median literacy {median_literacy}")

imputed_value_count['def_literacy'] = data['def_literacy'].isnull().sum()

data['def_literacy'] = data['def_literacy'].fillna(median_literacy)
assert data['def_literacy'].notnull().all()


# ==========

print("17 - Checking: def_religion, def_religion_pp")
data = data.drop(columns=['def_religion'])

# Clarify the religion values
religion_clarified = {
    'P': 'Protestant',
    'RC': 'Roman Catholic',
    'INDEP.': 'Independent',
    'WESLEYAN': 'Wesleyan Methodist',
    'JEW': 'Jewish',
    'METHODIST': 'Methodist',
    'PRESB': 'Presbyterian',
    'PRESBYTERIAN': 'Presbyterian',
    'QUAKER': 'Quaker',
    'BAPTIST': 'Baptist',
    'LUTHERAN': 'Lutheran',
    'UNITARIAN': 'Unitarian',
    'DISSENTER': 'Dissenter',
    'CE': 'Church of England',
    'CHURCH OF ENGLAND': 'Church of England',
    'PAGAN': 'Pagan',
    'MUSLIM': 'Muslim',
    'R': 'UNKNOWN',
    'NO DATA': 'UNKNOWN',
    'UNKNOWN': 'UNKNOWN',
    np.nan: 'UNKNOWN',
}

data['def_religion_pp'] = data['def_religion_pp'].map(religion_clarified)
assert data['def_religion_pp'].notnull().all()

# Check the number of UNKNOWN values in the def_religion_pp column
unknown_count = data['def_religion_pp'].value_counts().get('UNKNOWN', 0)
imputed_value_count['def_religion_pp'] = unknown_count

# ==========

print("18 - Checking: marital_status, marital_status_pp")
data = data.drop(columns=['marital_status'])
# Replace all null values with the mode value
print(f"    {data['marital_status_pp'].isnull().sum()} rows with null marital_status_pp values, replaced with the mode value UNKNOWN")
data['marital_status_pp'] = data['marital_status_pp'].fillna('UNKNOWN')

# Check the number of UNKNOWN values in the marital_status_pp column
unknown_count = data['marital_status_pp'].value_counts().get('UNKNOWN', 0)
imputed_value_count['marital_status_pp'] = unknown_count

# ==========

print("19 - Checking: children_nr")
# Try to convert all values in the 'children_nr' column to numeric. If it fails, replace the value with null
data['children_nr'] = pd.to_numeric(data['children_nr'], errors='coerce')
median_value = data['children_nr'].median()
# Replace all null values with the median value
print(f"    {data['children_nr'].isnull().sum()} rows with null children_nr values, replaced with the median value {median_value}")

imputed_value_count['children_nr'] = data['children_nr'].isnull().sum()

data['children_nr'] = data['children_nr'].fillna(median_value)
assert data['children_nr'].notnull().all()

# ==========

print("20 - Checking: cash_sav, cash_sav_pp")
data = data.drop(columns=['cash_sav'])
# Try to convert all values in the 'cash_sav_pp' column to numeric. If it fails, replace the value with null
data['cash_sav_pp'] = pd.to_numeric(data['cash_sav_pp'], errors='coerce')
median_value = data['cash_sav_pp'].median()
# Replace all null values with the median value
print(f"    {data['cash_sav_pp'].isnull().sum()} rows with null cash_sav_pp values, replaced with the median value {median_value}")

imputed_value_count['cash_sav_pp'] = data['cash_sav_pp'].isnull().sum()

data['cash_sav_pp'] = data['cash_sav_pp'].fillna(median_value)
assert data['cash_sav_pp'].notnull().all()
# ==========

print("21 - Checking: occupation, occupation_pp")
# Drop the original occupation column as it has been preprocessed
data = data.drop(columns=['occupation'])

# Only process the top 30 most common occupations, setting others to 'UNKNOWN'

# Replace all null values with 'UNKNOWN'
data['occupation_pp'] = data['occupation_pp'].replace({np.nan: 'UNKNOWN'})

most_common_occupations = data['occupation_pp'].value_counts()
for occupation, count in most_common_occupations.items():
    if count < 30:
        data['occupation_pp'] = data['occupation_pp'].replace({occupation: 'UNKNOWN'})

def clean_occupation(val):
    if val in ['FARM', 'FARMER\'S', 'PLOUGHMAN']:
        return 'FARM LABOURER'
    elif val in ['HOUSE', 'INDOOR', 'SERVANT']:
        return 'DOMESTIC'
    elif val == 'STONE':
        return 'STONEMASON'
    elif val == 'BRICK':
        return 'BRICKLAYER'
    elif val == 'IMPERFECT':
        return 'UNKNOWN'
    elif val == 'SHOE':
        return 'SHOEMAKER'
    else:
        return val

data['occupation_pp'] = data['occupation_pp'].apply(clean_occupation)


# Count all UNKNOWN values
unknown_count = data['occupation_pp'].value_counts().get('UNKNOWN', 0)
print(f"    Number of UNKNOWN values in occupation_pp: {unknown_count}")

imputed_value_count['occupation_pp'] = unknown_count

assert data['occupation_pp'].notnull().all()

# ==========

print("22 - Dropped Columns: birth_country")
# Count the number of unique birth countries
unique_birth_countries = data['birth_country'].nunique()
print(f"    Number of unique birth countries: {unique_birth_countries}")
data = data.drop(columns=['birth_country'])

# ==========

print("23 - Checking: coloffence_info")

# Map the values to boolean
def coloffence_info_mapping(val):
    if val in ['.', np.nan]:
        return False
    else:
        return True
    
data['coloffence_info'] = data['coloffence_info'].apply(coloffence_info_mapping)

imputed_value_count['coloffence_info'] = 0

# ==========
print("24 - Dropped Columns: prev_transp")
# Unique values in prev_transp: [nan 'Y' 'AUSTRALIA']
data = data.drop(columns=['prev_transp'])

# ==========

print("25 - Dropped Columns: priorconvictions_text")
data = data.drop(columns=['priorconvictions_text'])
# This column is very important for the analysis, but we don't have enough NLP tools available to process it

# ==========

print("26 - Dropped Columns: depni_date, depni_ship, arrivni_date")
data = data.drop(columns=['depni_date'])
data = data.drop(columns=['depni_ship'])
data = data.drop(columns=['arrivni_date'])
# Too many null values in arriveni date, cannot calculate the length of stay

# ==========
print("27 - Checking: offence_ni")

# NO DATA: 838
# NONE: 688
# NOTHING RECORDED: 175
# COMMITTED FOR TRIAL 1834 RIOT: 47
# NIL: 47

# For all values = 'NO DATA', 'NONE', 'NOTHING RECORDED', 'COMMITTED FOR TRIAL 1834 RIOT', 'NIL', replace false
# Otherwise, replace with True
def offence_ni_mapping(val):
    if val in ['NO DATA', 'NONE', 'NOTHING RECORDED', 'COMMITTED FOR TRIAL 1834 RIOT', 'NIL', np.nan]:
        return False
    else:
        return True
data['offence_ni'] = data['offence_ni'].apply(offence_ni_mapping)
assert data['offence_ni'].notnull().all()

imputed_value_count['offence_ni'] = 0

# =========
print("28 - Checking: death_notes")
data = data.drop(columns=['death_notes'])
# Convert all null into False
data["death_in_custody_pp"] = data["death_in_custody_pp"].replace({np.nan: False})
# Check it contains only True or False
if data["death_in_custody_pp"].isin([True, False]).all():
    print("    death_in_custody_pp contains only True or False")

imputed_value_count['death_in_custody_pp'] = 0

# ==========
print("29 - Dropped Columns: probat_dur")
# This column is too messy
data = data.drop(columns=['probat_dur'])

# ==========

# Using the columns date_arrived_aus, probat_date, and tl_date
# to calculate the length of stay until probation and getting the ticket of leave

print("30 - Checking: date_arrived_aus, probat_date, tl_date")
# Convert to datetime
data['date_arrived_aus'] = pd.to_datetime(data['date_arrived_aus'], errors='coerce')
data['probat_date'] = pd.to_datetime(data['probat_date'], errors='coerce')
data['tl_date'] = pd.to_datetime(data['tl_date'], errors='coerce')
# Create a new column 'length_of_stay_until_probat' with the difference between probat_date and date_arrived_aus
data['length_of_stay_until_probat'] = (data['probat_date'] - data['date_arrived_aus']).dt.days / 365
# Create a new column 'length_of_stay_until_tl' with the difference between tl_date and date_arrived_aus
data['length_of_stay_until_tl'] = (data['tl_date'] - data['date_arrived_aus']).dt.days / 365

# Set any negative values to null
data['length_of_stay_until_probat'] = data['length_of_stay_until_probat'].where(data['length_of_stay_until_probat'] >= 0, np.nan)
data['length_of_stay_until_tl'] = data['length_of_stay_until_tl'].where(data['length_of_stay_until_tl'] >= 0, np.nan)

# Print the number of null values
null_length_of_stay_until_probat_count = data['length_of_stay_until_probat'].isnull().sum()
null_length_of_stay_until_tl_count = data['length_of_stay_until_tl'].isnull().sum()
print(f"    {null_length_of_stay_until_probat_count} rows with null length_of_stay_until_probat values")
print(f"    {null_length_of_stay_until_tl_count} rows with null length_of_stay_until_tl values")

# Median values
median_length_of_stay_until_probat = data['length_of_stay_until_probat'].median()
median_length_of_stay_until_tl = data['length_of_stay_until_tl'].median()

imputed_value_count['length_of_stay_until_probat'] = null_length_of_stay_until_probat_count
imputed_value_count['length_of_stay_until_tl'] = null_length_of_stay_until_tl_count

# Replace null values with the median value
data['length_of_stay_until_probat'] = data['length_of_stay_until_probat'].fillna(median_length_of_stay_until_probat)
data['length_of_stay_until_tl'] = data['length_of_stay_until_tl'].fillna(median_length_of_stay_until_tl)

assert data['length_of_stay_until_probat'].notnull().all()
assert data['length_of_stay_until_tl'].notnull().all()

# Drop the original date_arrived_aus, probat_date, and tl_date columns
data = data.drop(columns=['date_arrived_aus'])
data = data.drop(columns=['probat_date'])
data = data.drop(columns=['tl_date'])

# ==========

print("31 - Checking: trial_court, date_1stkconv, offence_1stkconv, trialplace_1stkconv, sentence_1stkconv, sentduryears_1stkconv, court_1stkconv")

# Drop all columns except date_1stkconv
data = data.drop(columns=['trial_court'])
data = data.drop(columns=['offence_1stkconv'])
data = data.drop(columns=['trialplace_1stkconv'])
data = data.drop(columns=['sentence_1stkconv'])
data = data.drop(columns=['sentduryears_1stkconv'])
data = data.drop(columns=['court_1stkconv'])

# Convert date_1stkconv to datetime
data['date_1stkconv'] = pd.to_datetime(data['date_1stkconv'], errors='coerce')

# A function that map date_1stkconv to whether the convect has previous convictions or not 
# by comparing the date_1stkconv with the month_1stkconv and year_1stkconv
null_row = 0
true_row = 0
false_row = 0
def map_previous_convictions(row):
    global null_row, true_row, false_row
    # If either are null, return False
    if pd.isnull(row['date_1stkconv']) or pd.isnull(row['trial_month']) or pd.isnull(row['trial_year']):
        null_row += 1
        return False
    if row['date_1stkconv'] < pd.to_datetime(f"{row['trial_year']}-{row['trial_month']}-01"):
        true_row += 1
        return True
    else:
        false_row += 1
        return False
# Apply the function to create a new column
data['previous_convictions'] = data.apply(map_previous_convictions, axis=1)
# Drop the original date_1stkconv column
data = data.drop(columns=['date_1stkconv'])
print(f"    {null_row} convictions's previous convictions are UNKNOWN")
print(f"    {true_row} convictions has previous convictions")
print(f"    {false_row} convictions has no previous convictions")

imputed_value_count['previous_convictions'] = null_row
# ==========

1 - Checking: trial_id
    trial_id is unique and not null
2 - Dropped Columns: def_firstname, def_othername, def_surname
3 - Dropped Columns: register_name
4 - Dropped Columns: def_sex
5 - Dropped Columns: notes
6 - Checking: offence
    Number of unique offences in offence_pp: 205
    Number of unique offences in offence_pp: 36
    Number of unique offences in offence_pp_general: 10
    Unique offences in offence_pp_general:
        STEALING: 2490
        UNKNOWN: 1940
        ASSAULT: 667
        ROBBERY: 570
        SEXUAL: 203
        FORGERY: 172
        PROCESSING: 169
        DESTRUCTION: 125
        ESCAPE: 105
        FINANCE: 31
    Unique offences in offence_pp:
        UNKNOWN: 1940
        BURGLARY: 1252
        STEALING: 779
        HIGHWAY_ROBBERY: 411
        ASSAULT: 206
        CATTLE_STEALING: 190
        ROBBERY: 159
        RAPE: 153
        SHEEP_STEALING: 135
        FORGERY: 124
        HORSE_STEALING: 115
        ARSON: 105
        RECEIVING_STOLEN_GOODS: 100


In [11]:
display(data)  # Display the first few rows of the dataset

,trial_id,offence_pp_general,offence_pp,pp_sentence_years,trial_month,trial_year,trial_place,pris_ht_pp,def_age_pp,def_literacy,def_religion_pp,marital_status_pp,children_nr,cash_sav_pp,occupation_pp,coloffence_info,offence_ni,death_in_custody_pp,length_of_stay_until_probat,length_of_stay_until_tl,previous_convictions
0,695735,SEXUAL,RAPE,99.0,3,1844,UNKNOWN,165.10,25.0,2.0,Protestant,S,2.0,48.0,FARM LABOURER,False,False,False,3.824658,9.652055,False
1,695736,SEXUAL,RAPE,99.0,2,1844,UNKNOWN,165.10,44.0,2.0,Protestant,S,2.0,48.0,BOATMAN,False,True,False,3.824658,9.169863,False
2,695737,STEALING,BURGLARY,10.0,8,1843,UNKNOWN,167.64,31.0,3.0,Protestant,S,2.0,48.0,UNKNOWN,False,False,False,2.841096,7.795890,False
3,695738,STEALING,BURGLARY,10.0,3,1844,STAFFORD,170.18,49.0,2.0,Protestant,M,11.0,48.0,SHOEMAKER,False,False,False,2.841096,4.257534,False
4,695739,ASSAULT,CUTTING,15.0,3,1844,UNKNOWN,167.64,23.0,1.0,Protestant,S,2.0,48.0,FARM LABOURER,False,False,False,2.841096,7.795890,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6463,714850,STEALING,STEALING,99.0,8,1835,AUSTRALIA,165.10,23.0,2.0,UNKNOWN,UNKNOWN,2.0,48.0,SOLDIER,False,False,False,3.649315,7.795890,False
6465,714852,STEALING,HORSE_STEALING,10.0,7,1844,AUSTRALIA,165.10,27.0,3.0,Roman Catholic,S,2.0,48.0,FARM LABOURER,False,False,False,3.649315,7.795890,False
6466,714853,STEALING,BURGLARY,15.0,2,1840,AUSTRALIA,165.10,26.0,3.0,Roman Catholic,S,2.0,48.0,UNKNOWN,False,True,False,3.649315,7.795890,False
6468,714855,ROBBERY,HIGHWAY_ROBBERY,3.0,5,1833,AUSTRALIA,165.10,18.0,2.0,Roman Catholic,S,2.0,48.0,UNKNOWN,False,False,False,3.649315,7.795890,False


In [12]:
# Check all columns are not null
assert data.notnull().all().all()

# For each columns, if it is NOMINAL, count the number of unique values
# If it is NUMERIC, show the range, mean, median, mode, and standard deviation
for i, column in enumerate(data.columns):
    print(f"Column {i + 1}: {column}, Imputed Value Count: {imputed_value_count.get(column)}, Percentage: {imputed_value_count.get(column) / len(data) * 100:.2f}%")
    if data[column].dtype == 'object':
        print(f"    Number of unique values: {data[column].nunique()}")
        # Print all unique values along with their counts
        print("    Unique values:")
        for value, count in data[column].value_counts().items():
            print(f"        {value}: {count}")
        # Number of UNKNOWN values
        unknown_count = data[column].value_counts().get('UNKNOWN', 0)
        print(f"    Imputed Value Count: {unknown_count}")
    else:
        print(f"    Range: {data[column].min()} - {data[column].max()}")
        print(f"    Mean: {data[column].mean()}")
        print(f"    Median: {data[column].median()}")
        print(f"    Mode: {data[column].mode()[0]}")
        print(f"    Standard deviation: {data[column].std()}")

Column 1: trial_id, Imputed Value Count: 0, Percentage: 0.00%
    Range: 695735 - 714857
    Mean: 704675.7345692016
    Median: 699089.0
    Mode: 695735
    Standard deviation: 7691.5461735420695
Column 2: offence_pp_general, Imputed Value Count: 1940, Percentage: 40.87%
    Number of unique values: 10
    Unique values:
        STEALING: 2429
        ASSAULT: 639
        ROBBERY: 564
        UNKNOWN: 367
        SEXUAL: 198
        FORGERY: 165
        PROCESSING: 160
        DESTRUCTION: 122
        ESCAPE: 73
        FINANCE: 30
    Imputed Value Count: 367
Column 3: offence_pp, Imputed Value Count: 1940, Percentage: 40.87%
    Number of unique values: 36
    Unique values:
        BURGLARY: 1241
        STEALING: 741
        HIGHWAY_ROBBERY: 406
        UNKNOWN: 367
        ASSAULT: 191
        CATTLE_STEALING: 182
        ROBBERY: 158
        RAPE: 148
        SHEEP_STEALING: 132
        FORGERY: 122
        HORSE_STEALING: 114
        ARSON: 103
        RECEIVING_STOLEN_GOODS: 

In [13]:
# Output to a new CSV file
data.to_csv('VDL_Norfolk_Island_Penal_Colony_Cleaned.csv', index=False)